In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Dependencies

In [ ]:
import random
import timeit
import numpy as np
import copy
import matplotlib.pyplot as plt
import pickle
!pip --version
!python --version
!pip list

# Change the dimension of the board here

In [ ]:
BM = 6 #6 for 6x6, 3 for 3x3

# Transform Dictionary

In [ ]:
#Transform the Tic - Tac - Toe board to 6 transformations
board_dim = BM
matrix = [[(index, elem%board_dim) for elem in val] for index,val in enumerate(np.array(range(board_dim**2)).reshape(board_dim, board_dim))]

def rotate(mat, N):
    # Consider all squares one by one 
    for x in range(0, int(N/2)):      
        # Consider elements in group    
        # of 4 in current square 
        for y in range(x, N-x-1):   
            # store current cell in temp variable 
            temp = mat[x][y] 
            # move values from right to top 
            mat[x][y] = mat[y][N-1-x] 
            # move values from bottom to right 
            mat[y][N-1-x] = mat[N-1-x][N-1-y] 
            # move values from left to bottom 
            mat[N-1-x][N-1-y] = mat[N-1-y][x] 
            # assign temp to left 
            mat[N-1-y][x] = temp 
    return mat
  
# Function to pr the matrix 
def displayMatrix(mat, N):  
    for i in range(0, N): 
        for j in range(0, N): 
            print (mat[i][j], end = ' ') 
        print ("") 

def anticlock_wise(input_matrix, board_dim, number=1):
    
    diagonal_flip = copy.deepcopy(input_matrix)
    anti_diagonal_flip = copy.deepcopy(input_matrix)
    matrix = copy.deepcopy(input_matrix)
    
    input_matrix_list = np.array(matrix).reshape(1,board_dim**2, 2)[0]
    transform_dictionary = {tuple(entry):[] for entry in input_matrix_list}
    
    #displayMatrix(matrix, board_dim)
    #print('\n')  
    
    for i in range(number):
        matrix = rotate(matrix,board_dim)
        new_matrix_list = np.array(matrix).reshape(1,board_dim**2, 2)[0]
        count=0
        for key in transform_dictionary:
            transform_dictionary[key].append(tuple(new_matrix_list[count]))
            count+=1
        #displayMatrix(matrix, board_dim)
        #print('\n')
        
    
    
    #displayMatrix(anti_diagonal_flip, board_dim)
    #print('\n')  
    
    
    for row in range(0, board_dim):
        for col in range(row):
            diagonal_flip[row][col],diagonal_flip[col][row] = diagonal_flip[col][row],diagonal_flip[row][col]
    
    #displayMatrix(diagonal_flip, board_dim)
    #print('\n')
    
    new_matrix_list = np.array(diagonal_flip).reshape(1,board_dim**2, 2)[0]
    count=0
    for key in transform_dictionary:
        transform_dictionary[key].append(tuple(new_matrix_list[count]))
        count+=1
            
    for row in range(0, board_dim):
        for col in range(board_dim-1-row):
            anti_diagonal_flip[row][col],anti_diagonal_flip[board_dim-1-col][board_dim-1-row] = anti_diagonal_flip[board_dim-1-col][board_dim-1-row],anti_diagonal_flip[row][col]
            
    #displayMatrix(anti_diagonal_flip, board_dim)
    #print('\n')
    

    new_matrix_list = np.array(anti_diagonal_flip).reshape(1,board_dim**2, 2)[0]
    count=0
    for key in transform_dictionary:
        transform_dictionary[key].append(tuple(new_matrix_list[count]))
        count+=1
    
    return transform_dictionary
transform_dict = anticlock_wise(matrix, board_dim, number=3) #anticlockwise_rotation (3) + diagonal flip
#transform_dict

# Import the previous learning

In [ ]:
with open("RL_6x6_Values.p", "rb") as f:
    Imported_values = pickle.load(f)
    #Imported_values = dict()

# RL Agent

In [ ]:
class RL_Agent(object):
    
    #Value updation happens here
    
    def __init__(self, board_dim=BM, Imported_values=Imported_values): 
        
        self.count = 0 #Beginning of the time
        self.board_dim = board_dim
        self.rl_board = np.array(['e']*(self.board_dim**2)).reshape(self.board_dim, self.board_dim)
        self.gamma = 0.9
        self.values = Imported_values#dict()
        self.start = None #Episode start
        self.score = None #Episode score card
        self.episode = []
        self.slot = 0 #Beginning of episode

        
    #Choose a slot
    #If the game is started by RL agent, a empty is chosen randomly
    #Otherwise, the next move is chosen such that next state is having the highest value 
    def play(self, last_move, start=None):
        if start:
            self.start = start
            cord_of_empty_x_y = np.where(self.rl_board == 'e')
            cord = list(zip(cord_of_empty_x_y[0],cord_of_empty_x_y[1])) 
            random_slot = cord[np.random.choice(range(len(cord)),1)[0]] #1st position is randomly picked
            self.rl_board[random_slot[0]][random_slot[1]] = 'x'
            self.episode.append(random_slot)
            self.slot+=1
            return random_slot
        else:
            self.slot+=2
            self.rl_board[last_move] = 'o'
            self.episode.append(last_move)
            next_slot = None
            if self.slot in self.values:
                board_string_format = ''.join([''.join(row) for row in self.rl_board])
                possible_next_states=[key for key in self.values[self.slot].keys() if [(i,j) for i,j in zip(board_string_format,key) if i!=j] == [('e', 'x')]]
                if possible_next_states:
                    next_state_string_format = sorted({state:self.values[self.slot][state] for state in possible_next_states}, key = lambda x: self.values[self.slot][x][0], reverse=True)[0]
                    if self.values[self.slot][next_state_string_format][0] > 0:
                        next_action = [i for i,ch in enumerate(board_string_format) if next_state_string_format[i]!=ch][0]
                        next_slot = (next_action//self.board_dim , next_action%self.board_dim)
                    else:
                        not_to_consider_states = {state for state in possible_next_states if self.values[self.slot][state][0]<0}
                        not_to_consider_states_next_action = [[i for i,ch in enumerate(board_string_format) if state[i]!=ch][0] for state in not_to_consider_states]
                        not_to_consider_states_next_slot = [(slot//self.board_dim, slot%self.board_dim) for slot in not_to_consider_states_next_action]
                        cord_of_empty_x_y = np.where(self.rl_board == 'e')
                        cord = list(zip(cord_of_empty_x_y[0],cord_of_empty_x_y[1]))
                        cord = list(set(cord).difference(set(not_to_consider_states_next_slot)))
                        if not cord:
                            next_action = [i for i,ch in enumerate(board_string_format) if next_state_string_format[i]!=ch][0]
                            next_slot = (next_action//self.board_dim , next_action%self.board_dim)
                        else:
                            next_slot = cord[np.random.choice(range(len(cord)),1)[0]]
                        #print("BINCY CHECK", 'possible_next_states', possible_next_states, 'self.values[self.slot]=', self.values[self.slot], 'not_to_consider_states_next_action = ', not_to_consider_states_next_action, 'not_to_consider_states_next_slot =', not_to_consider_states_next_slot, 'cord=', cord,  'next_slot=', next_slot)
            if next_slot:
                self.rl_board[next_slot[0]][next_slot[1]] = 'x'
                self.episode.append(next_slot)
                return next_slot
            else:
                cord_of_empty_x_y = np.where(self.rl_board == 'e')
                cord = list(zip(cord_of_empty_x_y[0],cord_of_empty_x_y[1])) 
                random_slot = cord[np.random.choice(range(len(cord)),1)[0]] #1st position is randomly picked
                self.rl_board[random_slot[0]][random_slot[1]] = 'x'
                self.episode.append(random_slot)
                return random_slot
                                
    #Get notified by the judge regarding the end of the game with the winner
    def notify(self, opp_move, score):
        if self.episode[-1]!=opp_move:
            self.episode.append(opp_move)
        self.score = score
        self.update_values()
        self.initialize_board()
    
    #Update the value function from the episode (x12 times learning)
    def update_values(self):
        
        
        #6x2 Transformations:
        for learn in ['SELF','OPP']:
            
            if learn == 'SELF':
                SCORE = self.score
                if self.start == 'x':
                    START = 'x'
                else:
                    START = 'o'
                
            else:
                
                if self.score == 'x':
                    SCORE = 'o'
                elif self.score == 'o':
                    SCORE = 'x'
                else:
                    SCORE = 'd'
                
                if self.start == 'x':
                    START = 'o'
                else:
                    START = 'x'
                    
                
            
            for i in range(6):
                if not i:
                    episode_transformed = self.episode
                else:
                    episode_transformed = list(map(lambda x:transform_dict[x][i-1], self.episode))
                board_transformed = np.array(['e']*(self.board_dim**2)).reshape(self.board_dim, self.board_dim)
                if START == 'o':
                    for (i,j) in episode_transformed[::2]:
                        board_transformed[i][j] = 'o'
                    for (i,j) in episode_transformed[1::2]:
                        board_transformed[i][j] = 'x'
                else:
                    for (i,j) in episode_transformed[::2]:
                        board_transformed[i][j] = 'x'
                    for (i,j) in episode_transformed[1::2]:
                        board_transformed[i][j] = 'o'

                #if SCORE != 'd':
                G = 0
                slot = len(episode_transformed)
                if SCORE == 'x':
                    reward = 1
                elif SCORE == 'd':
                    reward = -1
                else:
                    reward = -10
                for time in range(1,len(episode_transformed))[::-1]:
                    if board_transformed[episode_transformed[time][0]][episode_transformed[time][1]] == 'x':
                        G = ((self.gamma*G) + reward)
                        reward = 0
                        board_string_format = ''.join([''.join(row) for row in board_transformed])
                        if not slot in self.values:
                            self.values[slot] = {board_string_format:[G,1]}
                        elif not board_string_format in self.values[slot]:
                            self.values[slot][board_string_format] = [G,1]
                        else:
                            self.values[slot][board_string_format][1]+=1
                            self.values[slot][board_string_format][0]+=(G - self.values[slot][board_string_format][0])/self.values[slot][board_string_format][1]   

                    board_transformed[episode_transformed[time][0]][episode_transformed[time][1]] = 'e'
                    slot-=1
                #print(self.values)
                #print('\n')
    
    #To initialize the board and game after each episode
    def initialize_board(self):
        self.count +=1
        self.rl_board = np.array(['e']*(self.board_dim**2)).reshape(self.board_dim, self.board_dim)
        self.start = None
        self.score = None
        self.episode = []
        self.slot = 0 #Beginning of episode
        
        

In [ ]:
#Create an RL agent object and methods to play and notify
rl_agent1 = RL_Agent(BM)
rl_agent1_play = getattr(rl_agent1, 'play')
rl_agent1_notify = getattr(rl_agent1, 'notify')

# Random Player

In [ ]:
class Random_Player(object):
    def __init__(self, board_dim=BM):
        self.board_dim = board_dim
        self.rp_board = np.array(['e']*(self.board_dim**2)).reshape(self.board_dim, self.board_dim)
        self.start = None
        

    #Choose an empty slot with equal probability and update the last move
    def play(self, last_move, start=None):
        if start:
            self.start = start
        else:
            self.rp_board[last_move] = 'x'
        random_slot = self.random()
        self.rp_board[random_slot[0]][random_slot[1]] = 'o'
        return random_slot
        
    #Choose a random empty slot with equal probability
    def random(self):
        cord_of_empty_x_y = np.where(self.rp_board == 'e')
        cord = list(zip(cord_of_empty_x_y[0],cord_of_empty_x_y[1])) 
        random_slot = cord[np.random.choice(range(len(cord)),1)[0]] #1st position is randomly picked
        return random_slot
    
    #Dummy function; get notification from the judge
    def notify(self, last_move=None, score=None):
        self.initialize_board()
    
    #To initialize the board at the end of the game
    def initialize_board(self):
        self.rp_board = np.array(['e']*(self.board_dim**2)).reshape(self.board_dim, self.board_dim)
    

In [ ]:
#Create an random player object and methods to play and notify
random_player1 = Random_Player(BM)
random_player1_play = getattr(random_player1, 'play')
random_player1_notify = getattr(random_player1, 'notify')

# Judge

Judge's responsibility:
- Decide the starting player
- Regulate the game (whose turn when)
- Evaluate the winner
- Notify both the players about the end of the game and who is the winner
- Keep the statistics

In [ ]:
import numpy as np
class Judge(object):
    def __init__(self, board_dim=BM):
        self.board_dim = board_dim
        self.judge_board = np.array(['e']*(self.board_dim**2)).reshape(self.board_dim, self.board_dim)
        self.winning_pos = np.concatenate([[list(range(r, r+(self.board_dim*(self.board_dim-1))+1, self.board_dim)) for r in range(self.board_dim)] ,[list(range(c,c+self.board_dim)) for c in range(0,self.board_dim**2, self.board_dim)], [np.array(range(self.board_dim**2)).reshape(self.board_dim, self.board_dim).diagonal()], [np.array(range(self.board_dim-1,self.board_dim**2,self.board_dim))-range(self.board_dim)]])
        self.winning_pos_cord = np.array([[[wpe//self.board_dim, wpe%self.board_dim] for wpe in  wp] for wp in self.winning_pos])
        self.stat = [0, {'o': 0,'x':0, 'd':0 }]
        self.step = 0
        
    #For changing the board
    def change_board(self, board): #board should be np.array
        self.judge_board = board
        
    #To check if the game is done; Identifies the draw even if there are empty slots
    def isdone(self):
        #Isdone will be called only from 2*(board_dim)-1 moves
        winning_positions_status = [[self.judge_board[cord[0]][cord[1]] for cord in wps] for wps in self.winning_pos_cord]
        if ['x']*self.board_dim in winning_positions_status:
            return 'x'
        elif ['o']*self.board_dim in winning_positions_status:
            return 'o'
        elif [wp for wp in winning_positions_status if 'e' in wp]:
            return None
        else:
            return 'd' #Judge doensn't wait for the entire game but check if there is any scope of winning for anyone
    
    #To regulate the game
    #Alternative chances to each player
    #Identifies the end of the game
    #Notifies the players
    #Keep the statistics
    def game(self, start, number_of_games, method1_play, method2_play, method1_notify, method2_notify):
        for num in range(number_of_games):
            if start == 'o':
                check_iter = 1
            else:
                check_iter = 2
            score = self.isdone()
            last_move = None

            #self.print_board()
            while not score:
                if not self.step:
                    if check_iter%2:
                        last_move=method1_play(last_move, start) #Random player
                        self.judge_board[last_move] = 'o'
                    else:
                        last_move=method2_play(last_move, start) #RL Agent
                        self.judge_board[last_move] = 'x'
                    # HERE
                    #self.print_board()
                else:
                    if check_iter%2:
                        last_move=method1_play(last_move) #Random player
                        self.judge_board[last_move] = 'o'
                    else:
                        last_move=method2_play(last_move) #RL Agent
                        self.judge_board[last_move] = 'x'
                    # HERE
                    #self.print_board()
                self.step+=1
                #last_move is cordinate
                check_iter+=1
                score = self.isdone()
            self.step = 0 #Initialize
            self.update_stats(score)
            method2_notify(last_move, score) #RL Agent
            method1_notify(last_move, score) #Random player
            self.change_board(np.array(['e']*(self.board_dim**2)).reshape(self.board_dim, self.board_dim))
    
    #Update the statistics
    def update_stats(self, score):
        self.stat[0]+=1
        self.stat[1][score]+=1
    
    #To print the board
    def print_board(self):
        for i in range(self.board_dim): 
            for j in range(self.board_dim): 
                print(self.judge_board[i][j], end = " ") 
            print()
        print()

In [ ]:
#Create an judge object and methods to play and notify
judge = Judge(BM)
#Below for just one game
#judge.game('o',1, random_player1_play, rl_agent1_play, random_player1_notify, rl_agent1_notify)

# Check performance

In [ ]:
## Plot % of win, loss and draw for N games
## Alternate Turns for each player
win, loss, draw, total = [],[],[],[]

In [ ]:
### Multiples of 100 only
GAME=10000
x = range(0,GAME)

for iter in range(1,GAME+1):
    if iter%2:
        judge.game('x',1, random_player1_play, rl_agent1_play, random_player1_notify, rl_agent1_notify)
    else:
        judge.game('o',1, random_player1_play, rl_agent1_play, random_player1_notify, rl_agent1_notify)
    if not iter%100:
        win.append(judge.stat[1]['x'])
        loss.append(judge.stat[1]['o'])
        draw.append(judge.stat[1]['d'])
        total.append(judge.stat[0])

In [ ]:
sparse = 1
plt.plot(total[::sparse], win[::sparse], label='win', linewidth=3)
plt.plot(total[::sparse], loss[::sparse], label='loss', linewidth=3)
#plt.plot(total[::sparse], draw[::sparse], label='draw', linewidth=3)
plt.legend(loc='best')
plt.show()

In [ ]:
judge.stat

# Saving values to pickle file

In [ ]:
with open("RL_6x6_Values.p", "wb") as f:
    pickle.dump(rl_agent1.values, f)